In [ ]:
import requests
import tarfile
for i in ["2018","2019","2020"]:
    url = 'http://data.cquest.org/inpi_rncs/imr/stock/" + i + ".tgz'
    target_path = i + '.tgz'

    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(target_path, 'wb') as f:
            f.write(response.raw.read())

In [ ]:
import codecs
utf8reader = codecs.getreader('utf-8')
tar = tarfile.open("2018.tgz", "r:gz", encoding='utf-8')
for name in tar.getmembers():
    f = tar.extractfile(name)
    if f is not None:
         content = f.read()

In [ ]:
tar.extractall('./data.inpi')

In [ ]:
import functions as fc

In [ ]:
fc.import_tgz("2019", "./data2019")

In [ ]:
fc.import_tgz("2020", "./data2020")

In [ ]:
import os
from glob import glob
PATH = "data.inpi/"
EXT = "*.csv"
all_csv_files = [file
                 for path, subdir, files in os.walk(PATH)
                 for file in glob(os.path.join(path, EXT))]
print(all_csv_files)

In [ ]:

def import_all_csv(path2data):
    """_summary_

    Args:
        path (_type_): _description_
    """

    extension = "*.zip"
    all_csv_files = [file for path, subdir, files in os.walk(path2data) for \
                    file in glob(os.path.join(path, extension))]
    print(all_csv_files)
    list_df = []
    for filename in all_csv_files:
        print(filename)
        filesize = os.path.getsize(filename)
        if filesize != 0:
            temp_df = pd.read_csv(filename, sep=";")
            temp_df["file_path"] = filename
            list_df.append(temp_df)
    return list_df

In [ ]:
list_df = import_all_csv("data2019/")

In [ ]:
len(list_df)

In [ ]:
df.head()

In [ ]:
len(li)

In [ ]:
li[0].columns

In [ ]:
dfs = pd.concat(li)

In [ ]:
dfs.shape

In [ ]:
df.shape

In [ ]:
import s3fs
# Create filesystem object
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})
BUCKET_OUT = "radjerad"
FILE_KEY_OUT_S3 = "inpi/inpi_2018.csv"
FILE_PATH_OUT_S3 = BUCKET_OUT + "/" + FILE_KEY_OUT_S3

with fs.open(FILE_PATH_OUT_S3, 'w') as file_out:
    dfs.to_csv(file_out)